 ## Task
 
### Проаналізувати всі товари на сайті:  
    https://smallpacking.agrosem.ua/shop/

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

agrosem_link = "https://smallpacking.agrosem.ua/shop/"

def is_num(text):
    try:
        int(text)
        return True
    except ValueError:
        return False
    
def basic_request(url):
    response = requests.get(url)
    soup_result = BeautifulSoup(response.text, 'lxml')
    return soup_result


def get_next_page(current_page_link):
    soup = basic_request(current_page_link)
    page_numbers = soup.find('ul', class_='page-numbers')
    current_page = int(page_numbers.find('span', class_='page-numbers current').text)
    for page in page_numbers.find_all('a', class_='page-numbers'):
        if is_num(page.text):
            if int(page.text) == current_page + 1:
                next_page_link = page.get('href').strip('/')
                return next_page_link
    return None


def get_pages():
    pages_list = []
    current_page = agrosem_link
    while current_page is not None:
        pages_list.append(current_page)
        current_page = get_next_page(current_page)
    return pages_list


def parse_product(product_link):
    soup = basic_request(product_link)
    attributes_description = soup.find('div', class_='atributes-description')
    name = attributes_description.h2.text
    # print(name)
    box_type = attributes_description.find('span', class_='type-packaging-name').text
    # print(box_type)
    sku = attributes_description.find('span', class_='sku').text[5:]
    # print(sku)
    price = attributes_description.find('span', class_='regular-price').text.replace(' ', '')[:-3]
    # print(price, '\n------------------')
    return sku, name, box_type, price


def get_products_list():
    pages_list = get_pages()
    products_list = []
    for page in pages_list:
        soup = basic_request(page)
        products = soup.find('div', class_='product-category-wrapper')

        for product in products.find_all('div', class_='product-from-category-container'):
            description = product.a.get('href').strip('/')
            product_info = parse_product(description)
            products_list.append(product_info)
    return products_list


def sqlite_rec(res):
    crsr.executemany("INSERT OR REPLACE INTO Agro VALUES(?, ?, ?, ?);", res)
    connection.commit()
    
connection = sqlite3.connect("Agro.db")

crsr = connection.cursor()

crsr.execute("""CREATE TABLE IF NOT EXISTS Agro(
sku INT PRIMARY KEY,
product TEXT,
package TEXT,
price REAL);""")
connection.commit()
result = get_products_list()
sqlite_rec(result)


connection.close()
 
print('The database has been successfully created and filled with data')

The database has been successfully created and filled with data


### Summary

In [1]:
import sqlite3
connection = sqlite3.connect("Agro.db")
crsr = connection.cursor()

crsr.execute("SELECT * FROM Agro;")
records = crsr.fetchall()
print("Всьго на сайті https://smallpacking.agrosem.ua/shop/ представлено:", len(records), 'товарів')

list_prices = []
for record in records:
    list_prices.append(record[3])
    average = round(sum(list_prices)/len(list_prices))
    maximum = round(max(list_prices))
    minimum = round(min(list_prices))

print(f'\nСередня ціна за товар {average} грн\nМаксимальна ціна за товар {maximum} грн\nМінімальна ціна за товар {minimum} грн')
    
crsr.execute("SELECT * FROM Agro;")
first_5 = crsr.fetchmany(5)
print(f'\nПерші 5 рядків створеної бази данних:')
for record in first_5:
    print(f'{record}')


connection.close()

Всьго на сайті https://smallpacking.agrosem.ua/shop/ представлено: 119 товарів

Середня ціна за товар 6535 грн
Максимальна ціна за товар 56000 грн
Мінімальна ціна за товар 125 грн

Перші 5 рядків створеної бази данних:
(28164, 'Ferticare Hydro 6-14-30 + micro', 'Коробка', 1900.0)
(66353, 'Ferticare Kombi 14-11-25 + micro', 'Коробка', 1700.0)
(39566, 'MagPlon K (нітрат калію)', 'Коробка', 1200.0)
(69156, 'Pulan 34.4', 'Коробка', 7500.0)
(74266, 'Saletrosan®26', 'Коробка', 4400.0)
